In [1]:
using Revise

using Test
import TensorCrossInterpolation as TCI
import Random

In [2]:
using Test
import TensorCrossInterpolation as TCI
import TensorCrossInterpolation: rank, linkdims, TensorCI2, MultiIndex, evaluate, crossinterpolate2, pivoterror, tensortrain
import Random
import QuanticsGrids as QD

In [3]:
import TensorCrossInterpolation as TCI
using TensorCrossInterpolation
using Random
using Test
using ITensors
ITensors.disable_warn_order()

14

In [4]:
pivotsearch = :full
seed = 124

Random.seed!(seed)

R = 20
abstol = 1e-4
δ = 10.0 / 2^R # Peaks are wider than 1/2^R.
grid = QD.DiscretizedGrid{1}(R, (0.0,), (1.0,))

rindex = [rand(1:2, R) for _ in 1:100]

f(bitlist) = fx(QD.quantics_to_origcoord(grid, bitlist)[1])
rpoint = Float64[QD.quantics_to_origcoord(grid, r)[1] for r in rindex]

function fx(x)
    res = exp(-10 * x)
    for r in rpoint
        res += abs(x - r) < δ ? 2 * abstol : 0.0
    end
    res
end

localdims = fill(2, R)
firstpivot = ones(Int, R)
tci, ranks, errors = crossinterpolate2(
    Float64,
    f,
    localdims,
    [firstpivot];
    tolerance=abstol,
    maxbonddim=1000,
    maxiter=20,
    loginterval=1,
    verbosity=1,
    normalizeerror=false,
    pivotsearch=pivotsearch,
)
@show abstol

(length(extraouterIset), length(extraouterJset)) = (0, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extraouterIset), length(extraouterJset)) = (1, 1)
(length(extrao

0.0001

In [5]:
TCI.addglobalpivots2sitesweep!(
    tci, f, rindex,
    tolerance=abstol,
    normalizeerror=false,
    maxbonddim=1000,
    pivotsearch=:full,
    verbosity=1
)

(length(extraouterIset), length(extraouterJset)) = (0, 544)
(length(extraouterIset), length(extraouterJset)) = (2, 531)
(length(extraouterIset), length(extraouterJset)) = (4, 511)
(length(extraouterIset), length(extraouterJset)) = (8, 481)
(length(extraouterIset), length(extraouterJset)) = (16, 440)
(length(extraouterIset), length(extraouterJset)) = (30, 400)
(length(extraouterIset), length(extraouterJset)) = (55, 360)
(length(extraouterIset), length(extraouterJset)) = (88, 312)
(length(extraouterIset), length(extraouterJset)) = (130, 265)
(length(extraouterIset), length(extraouterJset)) = (186, 197)
(length(extraouterIset), length(extraouterJset)) = (233, 136)
(length(extraouterIset), length(extraouterJset)) = (279, 97)
(length(extraouterIset), length(extraouterJset)) = (326, 62)
(length(extraouterIset), length(extraouterJset)) = (375, 32)
(length(extraouterIset), length(extraouterJset)) = (427, 16)
(length(extraouterIset), length(extraouterJset)) = (469, 8)
(length(extraouterIset), l

0

In [10]:
    tt = TCI.TensorTrain(TCI.sitetensors(tci, f))
    @show TCI.linkdims(tt)
    println("Error on pivots:")
    for b in 1:length(tci)-1
        Iset = TCI.Iset(tci, b+1)
        Jset = TCI.Jset(tci, b)
        diff = maximum(abs, [TCI.evaluate(tt, vcat(i, j)) - f(vcat(i, j)) for i in Iset, j in Jset])
        println(b, "    ", diff, " ", diff > abstol)
    end
    println("Error on T:")
    for l in 1:length(tci)-1
        diff = [TCI.evaluate(tt, vcat(i, m, j)) - f(vcat(i, m, j)) for i in TCI.Iset(tci, l), j in TCI.Jset(tci, l), m in 1:localdims[l]]
        println(l, "    ", maximum(abs, diff), " ", maximum(abs, diff) > abstol)
    end

    println("Error on Π:")
    for b in 1:length(tci)-1
        Iset_b = TCI.Iset(tci, b)
        Jset_bp1 = TCI.Jset(tci, b+1)
        diff = [
            TCI.evaluate(tt, vcat(i, i1, i2, j)) - f(vcat(i, i1, i2, j)) for i in Iset_b, j in Jset_bp1, i1 in 1:localdims[b], i2 in 1:localdims[b+1]]
        println(b, "    ", maximum(abs, diff), " ", maximum(abs, diff) > abstol)
    end

(length.(Iset_), length.(Jset_)) = ([1, 2, 4, 8, 16, 28, 47, 66, 76, 87, 93, 91, 86, 78, 59, 32, 16, 8, 4, 2], [2, 4, 8, 16, 28, 47, 66, 76, 87, 93, 91, 86, 78, 59, 32, 16, 8, 4, 2, 1])
(length.(Iset_), length.(Jset_)) = ([1, 2, 4, 8, 16, 29, 48, 69, 81, 89, 96, 97, 95, 89, 67, 34, 16, 8, 4, 2], [2, 5, 9, 18, 33, 59, 87, 121, 149, 181, 188, 171, 121, 64, 32, 16, 8, 4, 2, 1])
Nesting info: Iset
  Nested: 1 < 2
  Nested: 2 < 3
  Nested: 3 < 4
  Nested: 4 < 5
  Nested: 5 < 6
  Nested: 6 < 7
  Nested: 7 < 8
  Nested: 8 < 9
  Nested: 9 < 10
  Nested: 10 < 11
  Nested: 11 < 12
  Nested: 12 < 13
  Nested: 13 < 14
  Nested: 14 < 15
  Nested: 15 < 16
  Nested: 16 < 17
  Nested: 17 < 18
  Nested: 18 < 19
  Nested: 19 < 20

Nesting info: Jset
  Nested: 2 < 1
  Nested: 3 < 2
  Nested: 4 < 3
  Nested: 5 < 4
  Nested: 6 < 5
  Nested: 7 < 6
  Nested: 8 < 7
  Nested: 9 < 8
  Nested: 10 < 9
  Nested: 11 < 10
  Nested: 12 < 11
  Nested: 13 < 12
  Nested: 14 < 13
  Nested: 15 < 14
  Nested: 16 < 15
  Nes

[2, 4, 8, 16, 28, 47, 66, 76, 87, 93, 91, 86, 78, 59, 32, 16, 8, 4, 2]
Error on pivots:
1    1.1102230246251565e-15 false
2    3.3306690738754696e-16 false
3    7.771561172376096e-16 false
4    7.771561172376096e-16 false
5    1.3322676295501878e-15 false
6    5.367687421142797e-5 false
7    5.367687421142797e-5 false
8    5.367687421142797e-5 false
9    0.00020000000000012716 true
10    0.00020000000000012716 true
11    0.00020000000000012716 true
12    0.00020000000000014863 true
13    0.00020000000000007544 true
14    0.00020000000000014863 true
15    0.00020000000000007544 true
16    2.7755575615628914e-16 false
17    1.6653345369377348e-16 false
18    2.7755575615628914e-17 false
19    8.673617379884035e-18 false
Error on T:
1    1.1102230246251565e-15 false
2    3.3306690738754696e-16 false
3    7.771561172376096e-16 false
4    7.771561172376096e-16 false
5    1.3322676295501878e-15 false
6    1.3322676295501878e-15 false
7    5.367687421142797e-5 false
8    5.367687421142797e-5 

In [7]:
@test sum(abs.([TCI.evaluate(tci, r) - f(r) for r in rindex]) .> abstol) == 0

Error During Test at /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X15sZmlsZQ==.jl:1
  Test threw exception
  Expression: sum(abs.([TCI.evaluate(tci, r) - f(r) for r = rindex]) .> abstol) == 0
  MethodError: no method matching evaluate(::TensorCI2{Float64}, ::Vector{Int64})
  
  Closest candidates are:
    evaluate(::TensorCrossInterpolation.Contraction{T}, ::AbstractVector{Int64}) where T
     @ TensorCrossInterpolation ~/git/TensorCrossInterpolation.jl/src/contraction.jl:179
    evaluate(::TensorCrossInterpolation.TTCache{V}, ::AbstractVector{Int64}; usecache, midpoint) where V
     @ TensorCrossInterpolation ~/git/TensorCrossInterpolation.jl/src/cachedtensortrain.jl:130
    evaluate(::TensorCrossInterpolation.AbstractTensorTrain{V}, ::Union{Tuple{Vararg{Int64, N}}, AbstractVector{Int64}}) where {N, V}
     @ TensorCrossInterpolation ~/git/TensorCrossInterpolation.jl/src/abstracttensortrain.jl:124
    ...
  
  Stacktrace:
  

Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")